In [1]:
# 数据处理
import pandas as pd
import numpy as np
import json
import csv
import random
import math
from itertools import product
from community import community_louvain



# 网络分析与可视化
import networkx as nx
from networkx.readwrite import json_graph
from pyvis import network as net
import matplotlib.pyplot as plt
import powerlaw # Power laws are probability distributions with the form:p(x)∝x−α

In [5]:
## 从CCFD语义特征数据中构建完整的Concept Network

# 读取完整语义特征数据

df_complete = pd.read_csv('01_Processed Data/Complete-Data.csv')
df_complete.head()

,Concept,Rel,Feature,Frequency,familiarity,concretness,上级类别,下级类别,coverage,cue_validity,categorical_distinctiveness
0,安全带,可以,可以-保护,27.096774,9.0,9.0,人造物,工具,0.851345,0.038320,0.158537
1,安全带,是,是-带子,22.258065,9.0,9.0,人造物,工具,0.851345,0.286019,0.012195
2,安全带,其他,其他-车,17.419355,9.0,9.0,人造物,工具,0.851345,0.122012,0.021341
3,安全带,是,是-安保的,16.451613,9.0,9.0,人造物,工具,0.851345,0.089938,0.041159
4,安全带,可以,可以-保护人,15.483871,9.0,9.0,人造物,工具,0.851345,0.149063,0.035061


In [ ]:
## 方法一【Archive】 ----------------------------------------
## 从完整网络中抽取概念关联网络 / 特征关联网络 （会运行很久）


# ## 抽取关联网络
# def Extract_Association_Network(df, entity_col, association_col, strength_thres):
#     Source = []
#     Target = []
#     Strength = []
    
#     entities = list(df[entity_col].unique())
    
#     for i in range(len(entities)):
#         for j in range(1, (len(entities) - i)):
#             listA = list(df[df[entity_col]==entities[i]][association_col].unique())
#             listB = list(df[df[entity_col]==entities[i+j]][association_col].unique())
#             strength = len(set(listA)&set(listB))
#             if strength > strength_thres:
#                 #print(features[i],features[i+j],features)
#                 Source.append(entities[i])
#                 Target.append(entities[i+j])
#                 Strength.append(strength)
                
#     data_AssociationNetwork = {'Source': Source, 'Target': Target, 'Strength': Strength}
#     df_AssociationNetwork = pd.DataFrame(data = data_AssociationNetwork)
#     return df_AssociationNetwork


# 抽取概念关联网络
# df_ConceptNetwork = Extract_Association_Network(df_complete, 'Concept', 'Feature', 0)
# df_ConceptNetwork.to_csv('02_Graph/Concept_Link.csv',index=False)

# 抽取特征关联网络
# df_FeatureNetwork = Extract_Association_Network(df_complete, 'Feature', 'Concept', 0)
# df_FeatureNetwork.to_csv('02_Grapha/SemanticFeature_Link.csv',index=False)

# 读取概念关联网络
# df_ConceptNetwork = pd.read_csv('02_Graph/Concept_Link.csv')

# # 将概念关联网络导入NetworkX
# G_ConceptNetwork = nx.Graph()
# for row in df_ConceptNetwork.iterrows():
#     G_ConceptNetwork.add_edge(row[1]['Source'], row[1]['Target'], weight=int(row[1]['Strength']))

In [9]:
## 方法二 ----------------------------------------
# 将CCFD研究中计算好的 Concept Matrix 导入NetworkX
input_data = pd.read_csv('00_Raw Data/CCFD_Concept-similarity-matrix.csv', index_col=0)
# input_data = pd.read_csv('00_Raw Data/concept_simi_norm_.csv', index_col=0)

G = nx.Graph(input_data.values)

# 给节点加上label
def make_label_dict(labels):
    l = {}
    for i, label in enumerate(labels):
        l[i] = label
    return l

with open('00_Raw Data/CCFD_Concept-similarity-matrix.csv', 'r', encoding='utf-8-sig') as f:
# with open('00_Raw Data/concept_simi_norm_.csv', 'r', encoding='utf-8-sig') as f:

    d_reader = csv.DictReader(f)
    headers = d_reader.fieldnames

labels=make_label_dict(headers)

G_ConceptNetwork = nx.relabel_nodes(G, labels)

# 加入分类标签
for node in G_ConceptNetwork.nodes:
    G_ConceptNetwork.nodes[node]['subcategory']=list(df_complete[df_complete['Concept']==node]['下级类别'].unique())[0]
    G_ConceptNetwork.nodes[node]['maincategory']=list(df_complete[df_complete['Concept']==node]['上级类别'].unique())[0]

# print("Number of Nodes:",len(G_ConceptNetwork.nodes))
# print("Number of Edges:",len(G_ConceptNetwork.edges))

In [10]:
# 导出文件

with open('02_Graph/ConceptNetwork_Complete_CCFD-Similarity-Matrix.json','w+') as f:
# with open('02_Graph/ConceptNetwork_Complete_Word2Vec.json','w+') as f:
    f.write(json.dumps(nx.node_link_data(G_ConceptNetwork), ensure_ascii=False))